This is the pytorch version of [this great work](https://www.kaggle.com/code/richolson/cmi-2025-1d-cnn-imu-only-baseline) with:
* preprocessing
* training
* validation

There are two models, one for IMU-only data, and the other for all-feature data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import gc
import pickle
import os
from pathlib import Path
from types import SimpleNamespace
import shutil

from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.amp import autocast, GradScaler
from torch.optim import lr_scheduler, Adam, AdamW

from glob import glob
from sklearn.model_selection import KFold, StratifiedGroupKFold
from metric import score

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
cfg = SimpleNamespace(**{})

cfg.fname = 'CMI-001'
cfg.seed = 42
cfg.device = torch.device('cuda')

cfg.input_path = Path('/kaggle/input/cmi-detect-behavior-with-sensor-data')
cfg.save_path = Path('./saved_models')

imc_features = ['acc_x', 'acc_y', 'acc_z']
all_features = [
    'acc_x', 'acc_y', 'acc_z', 'rot_w', 'rot_x', 'rot_y', 'rot_z', 
    'thm_1', 'thm_2', 'thm_3','thm_4', 'thm_5'
]
cfg.imc_index = [0,1,2,3,4,5,6]
cfg.all_index = [0,1,2,3,4,5,6,7,8,9,10,11] 
# choose which two feature to generate extra features
cfg.gen_index = [
    [0, 1],
    [0, 2],
    [1, 2]
]
# these sequences have only imc features
imc_only_seq_ids = ['SEQ_000389', 'SEQ_000993', 'SEQ_001905', 'SEQ_002559', 'SEQ_003071', 'SEQ_003221', 'SEQ_004536', 'SEQ_006554', 'SEQ_006816', 'SEQ_007924', 'SEQ_008584', 'SEQ_011032', 'SEQ_012168', 'SEQ_012170', 'SEQ_012619', 'SEQ_013381', 'SEQ_013964', 'SEQ_015093', 'SEQ_016489', 'SEQ_016690', 'SEQ_017041', 'SEQ_017397', 'SEQ_017550', 'SEQ_017726', 'SEQ_018069', 'SEQ_019829', 'SEQ_020153', 'SEQ_020185', 'SEQ_020377', 'SEQ_021335', 'SEQ_022409', 'SEQ_022726', 'SEQ_022927', 'SEQ_024987', 'SEQ_025240', 'SEQ_025685', 'SEQ_027154', 'SEQ_028775', 'SEQ_028871', 'SEQ_029693', 'SEQ_032705', 'SEQ_033061', 'SEQ_033922', 'SEQ_034359', 'SEQ_034994', 'SEQ_035090', 'SEQ_035116', 'SEQ_035309', 'SEQ_035419', 'SEQ_035699', 'SEQ_038052', 'SEQ_039253', 'SEQ_040910', 'SEQ_041113', 'SEQ_041199', 'SEQ_041541', 'SEQ_042079', 'SEQ_042145', 'SEQ_042187', 'SEQ_043744', 'SEQ_044004', 'SEQ_044143', 'SEQ_044475', 'SEQ_045927', 'SEQ_045995', 'SEQ_046244', 'SEQ_047721', 'SEQ_048233', 'SEQ_048511', 'SEQ_048861', 'SEQ_049663', 'SEQ_050020', 'SEQ_050738', 'SEQ_052603', 'SEQ_052680', 'SEQ_052722', 'SEQ_053884', 'SEQ_054261', 'SEQ_055297', 'SEQ_055346', 'SEQ_056071', 'SEQ_056418', 'SEQ_056430', 'SEQ_057193', 'SEQ_057886', 'SEQ_058311', 'SEQ_058580', 'SEQ_058729', 'SEQ_058753', 'SEQ_058923', 'SEQ_059569', 'SEQ_060041', 'SEQ_060392', 'SEQ_061589', 'SEQ_064578', 'SEQ_065179']

cfg.max_length = 188
cfg.lr = 1e-3
cfg.num_epochs = 75
cfg.batch_size = 128
cfg.workers = 4
cfg.grad_value = 2.0
cfg.accumulate = 1
cfg.decay = 0.01
cfg.no_decay = True

targets = [
    'Above ear - pull hair',
    'Cheek - pinch skin',
    'Eyebrow - pull hair',
    'Eyelash - pull hair',
    'Forehead - pull hairline',
    'Forehead - scratch',
    'Neck - pinch skin',
    'Neck - scratch',
    'Drink from bottle/cup',
    'Feel around in tray and pull out an object',
    'Glasses on/off',
    'Pinch knee/leg skin',
    'Pull air toward your face',
    'Scratch knee/leg skin',
    'Text on phone',
    'Wave hello',
    'Write name in air',
    'Write name on leg'
]
cfg.label2id = {}
for i, k in enumerate(targets):
    cfg.label2id[k] = i

cfg.num_classes = len(targets)
cfg.targets = np.array(targets)

random.seed(cfg.seed)
np.random.seed(cfg.seed)
torch.manual_seed(cfg.seed)
torch.cuda.manual_seed(cfg.seed)

## load data

In [3]:
demo = pd.read_csv(cfg.input_path / "train_demographics.csv")
train = pd.read_csv(cfg.input_path / "train.csv")

train_agg = train.groupby(['sequence_type','sequence_id','subject','gesture'])['acc_x'].agg(['count']).reset_index()
train_agg = train_agg.rename(columns={'count': 'seq_length'})
demo = pd.merge(train_agg, demo, on='subject', how='left').sort_values(['subject','sequence_id']).reset_index(drop=True)
print(f"We have {len(demo)} sequences for training.")

We have 8151 sequences for training.


In [4]:
kfold = StratifiedGroupKFold(n_splits=4, shuffle=True, random_state=42)
demo['fold'] = -1
layers = demo.sequence_type.astype(str) + '/' + demo.handedness.astype(str) + '/' + demo.sex.astype(str)
for i, (tr_idx, va_idx) in enumerate(kfold.split(demo, y=layers, groups=demo.subject)):
    demo.loc[va_idx, 'fold'] = i

# f_name is the file path for Dataset
demo['f_name'] = demo['subject'].astype(str) + '_' + demo['sequence_id'].astype(str)
demo.fold.value_counts()

fold
0    2190
1    2040
2    1984
3    1937
Name: count, dtype: int64

## calculate the mean/std for normalization

In [5]:
for fold in range(4):
    train_ids = set(demo.loc[demo.fold!=fold, "sequence_id"].unique())
    temp = train[train.sequence_id.isin(train_ids)]
    print(f"Fold {fold}, Size: {len(train_ids)}")
    all_features_mean = np.nanmean(temp[all_features], axis=0)
    all_features_std = np.nanstd(temp[all_features], axis=0)
    np.save(f"all_features_mean_fold{fold}.npy", all_features_mean)
    np.save(f"all_features_std_fold{fold}.npy", all_features_std)

Fold 0, Size: 5961
Fold 1, Size: 6111
Fold 2, Size: 6167
Fold 3, Size: 6214


## save input array for each sequence

In [6]:
file_path = Path("input_arrays")
if file_path.exists():
    shutil.rmtree(file_path)
file_path.mkdir(exist_ok=True)

for i, df in tqdm(train.groupby('sequence_id')):
    subject = df['subject'].values[0]
    sequence_id = df['sequence_id'].values[0]
    for col in all_features:
        df[col] = df[col].ffill().bfill().fillna(0.)
    f_name = f"{subject}_{sequence_id}.npy"
    np.save(file_path / f_name, df[all_features].values.astype(np.float32))

  0%|          | 0/8151 [00:00<?, ?it/s]

## some functions

In [7]:
def pad_sequence(arr, max_len):
    """
    pad sequences to a fixed length
    """
    target_shape = (max_len, arr.shape[1])
    if arr.shape[0] > max_len:
        return arr[:max_len, :]

    pad_length = max_len - arr.shape[0]
    padded_arr = np.zeros(target_shape, dtype=arr.dtype)
    padded_arr[:arr.shape[0], :] = arr
    return padded_arr


def feature_gen(arr, gen_index):
    """
    use two features to generate more features
    for example, acc_x - acc_y --> acc_x_y_diff
    """
    target_shape = (arr.shape[0], arr.shape[1]+len(gen_index))
    gen_arr = np.zeros(target_shape, dtype=arr.dtype)
    gen_arr[:, :arr.shape[1]] = arr
    for i, ind in enumerate(gen_index):
        gen_arr[:, arr.shape[1]+i] = arr[:, ind[0]] - arr[:, ind[1]]
    return gen_arr


def get_optimizer(model, cfg):
    no_decay = ["bias", "norm"]
    if cfg.no_decay:
        optimizer_parameters = [
            {'params': [p for n, p in model.named_parameters() 
                        if not any(nd in n for nd in no_decay)],
             'weight_decay': cfg.decay},
            {'params': [p for n, p in model.named_parameters() 
                        if any(nd in n for nd in no_decay)],
             'weight_decay': 0.0},
        ]
    else:
        optimizer_parameters = model.parameters()
    optimizer = torch.optim.AdamW(optimizer_parameters, lr=cfg.lr)
    return optimizer


def get_dataloader(dataset, cfg, istrain):
    dataloader = DataLoader(
        dataset, 
        batch_size=cfg.batch_size,
        num_workers=cfg.workers,
        shuffle=istrain,
        pin_memory=False,
        drop_last=istrain
    )
    return dataloader

In [8]:
class CMIDataset(Dataset):
    def __init__(self, df, cfg, feat_mean, feat_std, feature_index):
        self.cfg = cfg
        self.targets = df["sequence_type"].values
        self.labels = df["gesture"].values
        self.f_names = df["f_name"].values
        self.seq_lengths = df["seq_length"].values
        self.label2id = cfg.label2id
        self.feat_mean = feat_mean
        self.feat_std = feat_std
        self.feature_index = feature_index

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        inputs = np.load(f'/kaggle/working/input_arrays/{self.f_names[idx]}.npy')
        inputs = (inputs - self.feat_mean) / self.feat_std
        inputs = inputs[:, self.feature_index]
        seq_len = self.seq_lengths[idx]
        
        if len(inputs) > self.cfg.max_length:
            inputs = inputs[:self.cfg.max_length, :]
            seq_len = self.cfg.max_length
        if len(inputs) < self.cfg.max_length:
            inputs = pad_sequence(inputs, self.cfg.max_length)
            seq_len = self.cfg.max_length

        inputs = feature_gen(inputs, self.cfg.gen_index)
        return {
            'features': torch.from_numpy(inputs).float(),
            'targets': torch.tensor(self.label2id[self.labels[idx]], dtype=torch.long),
            'seq_len': torch.tensor(seq_len, dtype=torch.long)
        }


class CNN1D(nn.Module):
    def __init__(self, cfg, feature_dim):
        super().__init__()
        self.max_length = cfg.max_length
        self.feature_dim = feature_dim
        self.num_classes = cfg.num_classes

        self.block1 = nn.Sequential(
            nn.Conv1d(self.feature_dim, 256, kernel_size=7, padding=3),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Dropout(0.3)
        )
        self.block2 = nn.Sequential(
            nn.Conv1d(256, 384, kernel_size=5, padding=2),
            nn.BatchNorm1d(384),
            nn.GELU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Dropout(0.3)
        )
        self.block3 = nn.Sequential(
            nn.Conv1d(384, 512, kernel_size=3, padding=1),
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Dropout(0.4)
        )
        self._to_linear = None
        self._calculate_conv_output((self.feature_dim, self.max_length))

        self.classifier = nn.Sequential(
            nn.Linear(self._to_linear, 512),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.GELU(),
            nn.Dropout(0.4),
            nn.Linear(256, self.num_classes)
        )
    
    def _calculate_conv_output(self, shape):
        with torch.no_grad():
            sample = torch.zeros(1, *shape)
            sample = self.block1(sample)
            sample = self.block2(sample)
            sample = self.block3(sample)
            self._to_linear = sample.view(1, -1).size(1)
    
    def forward(self, x):
        # remember to switch seq_length and num_features in Pytorch!
        x = x.permute(0, 2, 1)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [9]:
def batch_to_device(batch, device):
    return {k:batch[k].to(device, non_blocking=True) for k in batch.keys() if k not in []}


def train_epoch(loader, model, optimizer, scheduler, scaler, device, cfg, epoch):
    model.train()
    model.zero_grad()
    loss_output = []
    grad_norm_l = []
    accumulate = cfg.accumulate

    for batch in iter(loader):
        input_dict = batch_to_device(batch, device)
        with autocast(device_type="cuda", enabled=True):
            targets = input_dict['targets']
            features = input_dict['features']
            logits = model(features)
            loss = cfg.loss_func(logits, targets)

        loss_output.append(loss.detach().cpu().item())
        scaler.scale(loss / cfg.accumulate).backward() 
        accumulate -= 1
        
        if accumulate == 0:
            current_scale = scaler.get_scale()
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_value_(model.parameters(), cfg.grad_value)
            scaler.step(optimizer)     
            scaler.update()
            optimizer.zero_grad()
            accumulate = cfg.accumulate

        scheduler.step()


def valid_epoch(loader, model, device, cfg):
    model.eval()
    model.zero_grad()
    preds_output = []
    targets_output = []
    with torch.no_grad():
        for batch in iter(loader):      
            input_dict = batch_to_device(batch, device)
            features = input_dict['features']
            with autocast(device_type="cuda", enabled=False):
                preds = model(features)
                    
            preds_output.append(preds.detach().cpu())
            targets = batch['targets']
            targets_output.append(targets)

    preds_output = torch.cat(preds_output, dim=0)
    targets_output = torch.cat(targets_output, dim=0)
    return preds_output.numpy(), targets_output.numpy()

## CMI training part 

In [10]:
cfg.loss_func = nn.CrossEntropyLoss()

record_oof = []
record_pred = []
feature_dim = len(cfg.imc_index) + len(cfg.gen_index)

for fold in [0, 1, 2, 3]:

    print(f"*************************** Fold {fold} ***************************")
    train_fold = demo[demo.fold != fold].reset_index(drop=True)
    valid_fold = demo[demo.fold == fold].reset_index(drop=True)
    
    train_size = len(train_fold)
    valid_size = len(valid_fold)
    print(f"train size: {train_size}, valid size: {valid_size}")
    df_oof = valid_fold[['sequence_id', 'gesture']].copy()
    df_oof['fold'] = fold
    df_pred = valid_fold[['sequence_id']].copy()
    df_pred['fold'] = fold
    
    feat_mean = np.load(f"/kaggle/working/all_features_mean_fold{fold}.npy")
    feat_std = np.load(f"/kaggle/working/all_features_std_fold{fold}.npy")
    
    valid_ds = CMIDataset(valid_fold, cfg, feat_mean, feat_std, cfg.imc_index)
    valid_dataloader = get_dataloader(valid_ds, cfg, istrain=False)

    model = CNN1D(cfg, feature_dim).to(cfg.device)
    optimizer = get_optimizer(model, cfg)
    scaler = GradScaler()
    scheduler = None
    result = None
    best_score = -1
    for epoch in tqdm(range(cfg.num_epochs), total=cfg.num_epochs):
        train_ds = CMIDataset(train_fold, cfg, feat_mean, feat_std, cfg.imc_index)
        train_dataloader = get_dataloader(train_ds, cfg, istrain=True)
        if scheduler is None:
            scheduler = lr_scheduler.OneCycleLR(
                optimizer,
                max_lr=cfg.lr,
                epochs=cfg.num_epochs,
                steps_per_epoch=len(train_dataloader),
                pct_start=0.07,
                anneal_strategy="cos",
                final_div_factor=0.02,
            )
        
        train_epoch(train_dataloader, model, optimizer, scheduler, scaler, cfg.device, cfg, epoch)
        preds, targets = valid_epoch(valid_dataloader, model, cfg.device, cfg)
        # from argmax to string label
        p_index = preds.argmax(1)
        df_pred['gesture'] = cfg.targets[p_index]
    
        result = score(
            df_oof[['sequence_id', 'gesture']], 
            df_pred[['sequence_id', 'gesture']],
            row_id_column_name='sequence_id'
        )
    
        if result > best_score:
            print(f"Epoch {epoch+1} [{result:.4f}]")
            save_path = f'cmi_model_fold{fold}.pt'
            checkpoint = {
                'model' : model.state_dict(),
                'fold' : fold,
                'best_score': result
            }
            torch.save(checkpoint, save_path)
            best_score = result

    print()
    record_oof.append(df_oof)
    record_pred.append(df_pred)
    
    del model, optimizer, scheduler, scaler, train_dataloader, valid_dataloader
    gc.collect()
    torch.cuda.empty_cache()

record_oof = pd.concat(record_oof, ignore_index=True)
record_pred = pd.concat(record_pred, ignore_index=True)
record_oof.to_csv("cmi_oof.csv", index=False)
record_pred.to_csv("cmi_pred.csv", index=False)

*************************** Fold 0 ***************************
train size: 5961, valid size: 2190


  0%|          | 0/75 [00:00<?, ?it/s]

Epoch 1 [0.4847]
Epoch 2 [0.5657]
Epoch 3 [0.5713]
Epoch 5 [0.6083]
Epoch 9 [0.6277]
Epoch 10 [0.6339]
Epoch 11 [0.6359]
Epoch 13 [0.6434]
Epoch 14 [0.6450]
Epoch 15 [0.6583]
Epoch 22 [0.6687]
Epoch 23 [0.6692]
Epoch 30 [0.6746]
Epoch 49 [0.6747]
Epoch 50 [0.6774]
Epoch 53 [0.6783]
Epoch 62 [0.6826]
Epoch 71 [0.6839]
Epoch 75 [0.6880]

*************************** Fold 1 ***************************
train size: 6111, valid size: 2040


  0%|          | 0/75 [00:00<?, ?it/s]

Epoch 1 [0.4817]
Epoch 2 [0.5476]
Epoch 3 [0.5684]
Epoch 4 [0.5970]
Epoch 6 [0.6254]
Epoch 8 [0.6289]
Epoch 9 [0.6387]
Epoch 10 [0.6389]
Epoch 11 [0.6476]
Epoch 15 [0.6563]
Epoch 16 [0.6656]
Epoch 18 [0.6670]
Epoch 19 [0.6752]
Epoch 23 [0.6771]
Epoch 32 [0.6838]
Epoch 41 [0.6881]
Epoch 54 [0.6937]
Epoch 72 [0.6937]

*************************** Fold 2 ***************************
train size: 6167, valid size: 1984


  0%|          | 0/75 [00:00<?, ?it/s]

Epoch 1 [0.4807]
Epoch 2 [0.5379]
Epoch 3 [0.5728]
Epoch 4 [0.5797]
Epoch 5 [0.5957]
Epoch 8 [0.6155]
Epoch 10 [0.6202]
Epoch 12 [0.6355]
Epoch 16 [0.6380]
Epoch 21 [0.6520]
Epoch 22 [0.6542]
Epoch 25 [0.6595]
Epoch 31 [0.6618]
Epoch 34 [0.6650]
Epoch 43 [0.6730]
Epoch 56 [0.6741]
Epoch 60 [0.6769]

*************************** Fold 3 ***************************
train size: 6214, valid size: 1937


  0%|          | 0/75 [00:00<?, ?it/s]

Epoch 1 [0.4719]
Epoch 2 [0.5449]
Epoch 3 [0.5564]
Epoch 5 [0.5733]
Epoch 6 [0.5747]
Epoch 7 [0.5821]
Epoch 8 [0.5825]
Epoch 9 [0.5925]
Epoch 10 [0.6018]
Epoch 14 [0.6150]
Epoch 15 [0.6336]
Epoch 22 [0.6342]
Epoch 32 [0.6437]
Epoch 34 [0.6478]
Epoch 46 [0.6541]
Epoch 58 [0.6615]
Epoch 65 [0.6642]



In [11]:
score(
    record_oof[['sequence_id', 'gesture']], 
    record_pred[['sequence_id', 'gesture']],
    row_id_column_name='sequence_id'
)

0.6721769093943366

## CMI+THM training part 

In [12]:
cfg.loss_func = nn.CrossEntropyLoss()

record_oof = []
record_pred = []
feature_dim = len(cfg.all_index) + len(cfg.gen_index)

for fold in [0, 1, 2, 3]:

    print(f"*************************** Fold {fold} ***************************")
    train_fold = demo[demo.fold != fold].reset_index(drop=True)
    valid_fold = demo[demo.fold == fold].reset_index(drop=True)
    
    train_size = len(train_fold)
    valid_size = len(valid_fold)
    print(f"train size: {train_size}, valid size: {valid_size}")
    df_oof = valid_fold[['sequence_id', 'gesture']].copy()
    df_oof['fold'] = fold
    df_pred = valid_fold[['sequence_id']].copy()
    df_pred['fold'] = fold
    
    feat_mean = np.load(f"/kaggle/working/all_features_mean_fold{fold}.npy")
    feat_std = np.load(f"/kaggle/working/all_features_std_fold{fold}.npy")
    
    valid_ds = CMIDataset(valid_fold, cfg, feat_mean, feat_std, cfg.all_index)
    valid_dataloader = get_dataloader(valid_ds, cfg, istrain=False)

    model = CNN1D(cfg, feature_dim).to(cfg.device)
    optimizer = get_optimizer(model, cfg)
    scaler = GradScaler()
    scheduler = None
    result = None
    best_score = -1
    for epoch in tqdm(range(cfg.num_epochs), total=cfg.num_epochs):
        train_ds = CMIDataset(train_fold, cfg, feat_mean, feat_std, cfg.all_index)
        train_dataloader = get_dataloader(train_ds, cfg, istrain=True)
        if scheduler is None:
            scheduler = lr_scheduler.OneCycleLR(
                optimizer,
                max_lr=cfg.lr,
                epochs=cfg.num_epochs,
                steps_per_epoch=len(train_dataloader),
                pct_start=0.07,
                anneal_strategy="cos",
                final_div_factor=0.02,
            )
        
        train_epoch(train_dataloader, model, optimizer, scheduler, scaler, cfg.device, cfg, epoch)
        preds, targets = valid_epoch(valid_dataloader, model, cfg.device, cfg)
        # from argmax to string label
        p_index = preds.argmax(1)
        df_pred['gesture'] = cfg.targets[p_index]
    
        result = score(
            df_oof[['sequence_id', 'gesture']], 
            df_pred[['sequence_id', 'gesture']],
            row_id_column_name='sequence_id'
        )
    
        if result > best_score:
            print(f"Epoch {epoch+1} [{result:.4f}]")
            save_path = f'cmithm_model_fold{fold}.pt'
            checkpoint = {
                'model' : model.state_dict(),
                'fold' : fold,
                'best_score': result
            }
            torch.save(checkpoint, save_path)
            best_score = result

    print()
    record_oof.append(df_oof)
    record_pred.append(df_pred)
    
    del model, optimizer, scheduler, scaler, train_dataloader, valid_dataloader
    gc.collect()
    torch.cuda.empty_cache()

record_oof = pd.concat(record_oof, ignore_index=True)
record_pred = pd.concat(record_pred, ignore_index=True)
record_oof.to_csv("cmithm_oof.csv", index=False)
record_pred.to_csv("cmithm_pred.csv", index=False)

*************************** Fold 0 ***************************
train size: 5961, valid size: 2190


  0%|          | 0/75 [00:00<?, ?it/s]

Epoch 1 [0.5209]
Epoch 2 [0.5844]
Epoch 3 [0.6044]
Epoch 4 [0.6274]
Epoch 5 [0.6340]
Epoch 6 [0.6493]
Epoch 7 [0.6636]
Epoch 9 [0.6671]
Epoch 10 [0.6844]
Epoch 12 [0.6857]
Epoch 13 [0.6878]
Epoch 16 [0.6950]
Epoch 17 [0.6952]
Epoch 18 [0.6977]
Epoch 19 [0.7015]
Epoch 21 [0.7107]
Epoch 24 [0.7179]
Epoch 28 [0.7208]
Epoch 39 [0.7248]
Epoch 44 [0.7282]
Epoch 48 [0.7283]

*************************** Fold 1 ***************************
train size: 6111, valid size: 2040


  0%|          | 0/75 [00:00<?, ?it/s]

Epoch 1 [0.4732]
Epoch 2 [0.5764]
Epoch 3 [0.6267]
Epoch 5 [0.6466]
Epoch 6 [0.6618]
Epoch 8 [0.6647]
Epoch 9 [0.6836]
Epoch 10 [0.6839]
Epoch 11 [0.6920]
Epoch 13 [0.6943]
Epoch 15 [0.7034]
Epoch 16 [0.7130]
Epoch 21 [0.7186]
Epoch 25 [0.7239]
Epoch 35 [0.7265]
Epoch 36 [0.7293]
Epoch 42 [0.7297]
Epoch 45 [0.7361]
Epoch 53 [0.7375]
Epoch 60 [0.7473]

*************************** Fold 2 ***************************
train size: 6167, valid size: 1984


  0%|          | 0/75 [00:00<?, ?it/s]

Epoch 1 [0.4817]
Epoch 2 [0.5410]
Epoch 3 [0.6150]
Epoch 4 [0.6377]
Epoch 5 [0.6416]
Epoch 7 [0.6556]
Epoch 8 [0.6617]
Epoch 9 [0.6706]
Epoch 11 [0.6803]
Epoch 12 [0.6866]
Epoch 16 [0.6928]
Epoch 23 [0.6977]
Epoch 24 [0.7068]
Epoch 28 [0.7079]
Epoch 32 [0.7121]
Epoch 34 [0.7209]
Epoch 37 [0.7211]
Epoch 50 [0.7222]
Epoch 53 [0.7293]

*************************** Fold 3 ***************************
train size: 6214, valid size: 1937


  0%|          | 0/75 [00:00<?, ?it/s]

Epoch 1 [0.5063]
Epoch 2 [0.5960]
Epoch 3 [0.6087]
Epoch 5 [0.6113]
Epoch 6 [0.6262]
Epoch 7 [0.6468]
Epoch 8 [0.6475]
Epoch 10 [0.6483]
Epoch 11 [0.6560]
Epoch 14 [0.6628]
Epoch 15 [0.6644]
Epoch 16 [0.6728]
Epoch 26 [0.6836]
Epoch 38 [0.6923]



In [13]:
score(
    record_oof[['sequence_id', 'gesture']], 
    record_pred[['sequence_id', 'gesture']],
    row_id_column_name='sequence_id'
)

0.7136355701889614

CV: (0.67883 + 0.71494) / 2 = 0.69688

The inference part might be few days later!